<html><head></head><body><font color = 'red'><strong>▶메타정보</strong></font><table border = '1'><tbody><tr><th scope = 'row' bgcolor = '#cccccc'>공공정보명</th><td>120상담정보</td><th scope = 'row' bgcolor = '#cccccc'>서비스명</th><td>서울시 다산콜센터 자주묻는질문 목록 조회</td></tr><tr><th scope = 'row' bgcolor = '#cccccc'>서비스설명</th><td colspan = '3'>서울시 및 자치구에서 수행하는 정책</td></tr></table></html>
  
  
    
* [오픈 API 이용안내> 이용안내 | 서울열린데이터광장](https://data.seoul.go.kr/together/guide/useGuide.do)
* https://data.seoul.go.kr/dataList/OA-1127/S/1/datasetView.do

## 라이브러리 로드

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

## 라이센스 확인

In [2]:
api_document_url = "https://data.seoul.go.kr/dataList/OA-1127/S/1/datasetView.do"
df_api_doc = pd.read_html(api_document_url)[2]
df_api_doc = df_api_doc.set_index(0).T
df_api_doc

,공개일자,최신수정일자,갱신주기,분류,원본시스템,저작권자,제공기관,제공부서,원본형태,제3저작권자,라이센스,관련 태그
1,2012.05.11,2020.04.03,"비정기(수시, 자료변경시)",일반행정,120 다산콜센터 상담시스템,서울특별시,120다산콜재단,시스템관리팀,DB,없음,저작권자표시(BY) : 이용이나 변경 및 2차적 저작물의 작성을 포함한 자유이용을 ...,"다산 , 콜센터 , 120 , 질문 , 민원"


In [3]:
df_api_doc.loc[1, "라이센스"]

'저작권자표시(BY) : 이용이나 변경 및 2차적 저작물의 작성을 포함한 자유이용을 허락합니다.'

## 수집할 URL

In [4]:
import auth_key

auth_key_val = auth_key.seoul_api_key()

In [5]:
faq_url = f"http://openAPI.seoul.go.kr:8088/{auth_key_val}/xml/SearchFAQService/1/999/"

```
<row>
<FAQ_TP>S</FAQ_TP>
<FAQ_SEQNO>289515</FAQ_SEQNO>
<FAQ_TP_NAME>서울시 업무매뉴얼</FAQ_TP_NAME>
<LCODE>22214335</LCODE>
<LCODE_NAME>수송및교통</LCODE_NAME>
<QUEST>차 없는 거리_종로</QUEST>
<UPDATE_YMDHMS>20200414143921</UPDATE_YMDHMS>
</row>
```

## HTTP 요청으로 목록 수집

In [6]:
result = requests.get(faq_url, verify=False)

## XML 데이터 해석

In [7]:
xmlsoup = bs(result.text, "lxml-xml")

In [8]:
row_list = [row for row in xmlsoup.select("row")]

In [9]:
row_list[0]

<row>
<FAQ_TP>F</FAQ_TP>
<FAQ_SEQNO>289427</FAQ_SEQNO>
<FAQ_TP_NAME>FAQ</FAQ_TP_NAME>
<LCODE>22214327</LCODE>
<LCODE_NAME>일반공공행정</LCODE_NAME>
<QUEST>[시ㆍ구정외 타기관 관련 상담] 고용노동부 [일자리 안정자금]</QUEST>
<UPDATE_YMDHMS>20190522102043</UPDATE_YMDHMS>
</row>

## 함수로 만들기

In [10]:
row = row_list[0]
content_list = []
def parse_row(row):
    FAQ_TP = row.select("FAQ_TP")
    FAQ_TP_NAME = row.select("FAQ_TP_NAME")
    LCODE = row.select("LCODE")
    LCODE_NAME = row.select("LCODE_NAME")
    QUEST = row.select("QUEST")
    return FAQ_TP, FAQ_TP_NAME, LCODE, LCODE_NAME, QUEST

In [11]:
# http://openapi.seoul.go.kr:8088/sample/xml/SearchDetailsFAQService/1/5/F/

* F : FAQ
* S : 서울시 업무매뉴얼
* J : 자치구 업무매뉴얼

## 내용수집

In [12]:
faq_no = 289435
detail_url = f"http://openapi.seoul.go.kr:8088/sample/xml/SearchDetailsFAQService/1/5/F/{faq_no}"

In [13]:
result = requests.get(detail_url, verify=False)
xmlsoup = bs(result.text, "lxml-xml")

In [14]:
xmlsoup.select("CODE")[0].get_text() == "INFO-000"

True

In [15]:
# xmlsoup.select("ANSWER")

In [16]:
xmlsoup.select("QUEST")

[<QUEST>서대문구립인조잔디구장 FAQ</QUEST>]

## 하나의 데이터프레임으로 만들기

In [17]:
# 목록과 내용을 하나의 데이터프레임으로 만들기


## 파일로 저장하기